## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Georgetown,MY,5.4112,100.3354,80.53,83,20,5.75,few clouds
1,1,Khatanga,RU,71.9667,102.5000,-1.86,97,99,10.25,overcast clouds
2,2,Faya,SA,18.3851,42.4509,61.32,74,6,3.47,clear sky
3,3,Lashio,MM,22.9333,97.7500,56.10,96,0,1.39,clear sky
4,4,Burnie,AU,-41.0667,145.9167,51.03,75,88,8.08,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip?"))
max_temp = float(input("What is the maximum temperature you would like for your trip?"))

What is the minimum temperature you would like for your trip?70
What is the maximum temperature you would like for your trip?80


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp)& \
                                      (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
13,13,Atuona,PF,-9.8000,-139.0333,78.04,77,6,18.66,clear sky
16,16,Nhulunbuy,AU,-12.2333,136.7667,77.61,88,100,4.61,light rain
17,17,Vao,NC,-22.6667,167.4833,75.34,87,45,13.71,scattered clouds
20,20,Vaini,TO,-21.2000,-175.2000,73.56,100,40,2.30,scattered clouds
23,23,Cidreira,BR,-30.1811,-50.2056,73.85,77,52,15.03,broken clouds
25,25,Hurghada,EG,27.2574,33.8129,72.86,44,39,13.18,scattered clouds
27,27,Souillac,MU,-20.5167,57.5167,79.18,73,40,2.30,scattered clouds
29,29,Rikitea,PF,-23.1203,-134.9692,75.13,75,87,15.86,overcast clouds
32,32,Bambous Virieux,MU,-20.3428,57.7575,79.05,73,40,2.30,scattered clouds
36,36,Carnarvon,AU,-24.8667,113.6333,73.47,60,0,9.22,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                146
City                   146
Country                145
Lat                    146
Lng                    146
Max Temp               146
Humidity               146
Cloudiness             146
Wind Speed             146
Current Description    146
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
13,Atuona,PF,78.04,clear sky,-9.8000,-139.0333,
16,Nhulunbuy,AU,77.61,light rain,-12.2333,136.7667,
17,Vao,NC,75.34,scattered clouds,-22.6667,167.4833,
20,Vaini,TO,73.56,scattered clouds,-21.2000,-175.2000,
23,Cidreira,BR,73.85,broken clouds,-30.1811,-50.2056,
25,Hurghada,EG,72.86,scattered clouds,27.2574,33.8129,
27,Souillac,MU,79.18,scattered clouds,-20.5167,57.5167,
29,Rikitea,PF,75.13,overcast clouds,-23.1203,-134.9692,
32,Bambous Virieux,MU,79.05,scattered clouds,-20.3428,57.7575,
36,Carnarvon,AU,73.47,clear sky,-24.8667,113.6333,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found.....skipping.")
        
        

Hotel not found.....skipping.
Hotel not found.....skipping.
Hotel not found.....skipping.
Hotel not found.....skipping.
Hotel not found.....skipping.


In [25]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df=hotel_df.dropna()
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
13,Atuona,PF,78.04,clear sky,-9.8000,-139.0333,Villa Enata
16,Nhulunbuy,AU,77.61,light rain,-12.2333,136.7667,East Arnhem Real Estate
17,Vao,NC,75.34,scattered clouds,-22.6667,167.4833,Hotel Kou-Bugny
20,Vaini,TO,73.56,scattered clouds,-21.2000,-175.2000,Keleti Beach Resort
23,Cidreira,BR,73.85,broken clouds,-30.1811,-50.2056,Hotel Castelo


In [26]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [27]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp}°F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [28]:
# 11a. Add a marker layer for each city to the map. 
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))